# Рабочая отправка изображений, работает через vlc

In [1]:
import cv2
import os
from flask import Flask, Response

# Конфигурация сервера
host_name = '0.0.0.0'
port = 8000

# Путь к папке с изображениями
image_folder = r'C:\Users\PC1\Desktop\protocol\received_images'

# Получаем список всех файлов в папке
image_files = [f for f in os.listdir(image_folder) if os.path.isfile(os.path.join(image_folder, f))]
image_index = 0  # Индекс текущего изображения
#fps = 1 / 5  # Частота обновления изображений (кадр каждые 5 секунд)
resize_width = 1920  # Ширина изображения после изменения размера
resize_height = 1280  # Высота изображения после изменения размера

app = Flask(__name__)

def generate_video():
    global image_index
    while True:
        # Получаем путь к текущему изображению
        image_path = os.path.join(image_folder, image_files[image_index])

        # Чтение изображения с помощью OpenCV
        img = cv2.imread(image_path)

        if img is None:
            print(f"Ошибка: не удалось прочитать изображение {image_path}")
            continue

        # Изменение размера изображения
        print(f"Изменение размера изображения: {image_path} до {resize_width}x{resize_height}")
        img_resized = cv2.resize(img, (resize_width, resize_height))

        # Конвертация изображения в формат JPEG
        ret, jpeg = cv2.imencode('.jpg', img_resized)

        if not ret:
            print("Ошибка: не удалось конвертировать изображение в JPEG")
            continue

        # Отправка изображения в виде MJPEG кадра
        print(f"Отправка изображения: {image_path}")
        yield (b'--frame\r\n'
               b'Content-Type: image/jpeg\r\n\r\n' + jpeg.tobytes() + b'\r\n\r\n')

        # Обновляем индекс для следующего изображения
        image_index = (image_index + 1) % len(image_files)

        # Пауза для поддержания заданной частоты кадров
        cv2.waitKey(int(1000))

@app.route('/')
def video_feed():
    return Response(generate_video(), mimetype='multipart/x-mixed-replace; boundary=frame')

if __name__ == '__main__':
    print("Запуск скрипта")
    if not image_files:
        print(f"Ошибка: не найдены изображения в папке {image_folder}")
    else:
        print(f"Найдено {len(image_files)} изображений для трансляции")
        app.run(host=host_name, port=port)


Запуск скрипта
Найдено 35 изображений для трансляции
 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:8000
 * Running on http://192.168.0.99:8000
Press CTRL+C to quit
